In [5]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [6]:
train_dir = '/content/drive/MyDrive/VIPERC/dataset/train'
test_dir = '/content/drive/MyDrive/VIPERC/dataset/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5025 images belonging to 13 classes.
Found 1256 images belonging to 13 classes.


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

for layer in base_model.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 0s 0us/step


In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [11]:
model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, early_stopping])

Epoch 1/80
158/158 [==============================] - 39s 203ms/step - loss: 2.2625 - accuracy: 0.2537 - val_loss: 2.0020 - val_accuracy: 0.3296
Epoch 2/80
158/158 [==============================] - 28s 179ms/step - loss: 1.8266 - accuracy: 0.3910 - val_loss: 1.7494 - val_accuracy: 0.4371
Epoch 3/80
158/158 [==============================] - 29s 182ms/step - loss: 1.6173 - accuracy: 0.4627 - val_loss: 1.5439 - val_accuracy: 0.5279
Epoch 4/80
158/158 [==============================] - 29s 182ms/step - loss: 1.4238 - accuracy: 0.5319 - val_loss: 1.3923 - val_accuracy: 0.5502
Epoch 5/80
158/158 [==============================] - 28s 174ms/step - loss: 1.3231 - accuracy: 0.5705 - val_loss: 1.4231 - val_accuracy: 0.5279
Epoch 6/80
158/158 [==============================] - 29s 184ms/step - loss: 1.2342 - accuracy: 0.6090 - val_loss: 1.2138 - val_accuracy: 0.6131
Epoch 7/80
158/158 [==============================] - 29s 185ms/step - loss: 1.1774 - accuracy: 0.6304 - val_loss: 1.1584 - val_ac

In [12]:
!mv /content/best_model.keras /content/drive/MyDrive/VIPERC/

In [13]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/VIPERC/best_model.keras')

In [14]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

40/40 [==============================] - 8s 149ms/step - loss: 0.5289 - accuracy: 0.8511

Test Loss: 0.5289
Test Accuracy: 85.11%
40/40 [==============================] - 6s 123ms/step
Accuracy: 85.11146496815286
Precision: 86.6121120752765
Recall: 85.11146496815286
F1 Score: 84.32842798364636

Classification Report:
              precision    recall  f1-score   support

 cong_chieng     0.8788    0.9667    0.9206        60
     dan_bau     0.9744    0.9620    0.9682        79
      dan_co     1.0000    0.3404    0.5079        47
      dan_da     0.9697    0.7805    0.8649       123
     dan_day     0.8696    0.8696    0.8696        92
  dan_nguyet     0.8148    0.8800    0.8462       125
     dan_sen     0.9333    0.8550    0.8924       131
  dan_t_rung     0.8737    0.9708    0.9197       171
    dan_tinh     0.7933    0.9597    0.8686       124
   dan_tranh     0.7169    0.9573    0.8198       164
   dan_ty_ba     0.9231    0.5000    0.6486        48
        khen     0.8621    0.500